### Data Gathering Quickstart
This notebook demonstrates how to use the `DataGatherer` to fetch financial data from various sources.

In [ ]:
from jarjarquant.data_gatherer import DataGatherer

import sys
import os
import nest_asyncio
nest_asyncio.apply()

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

dg = DataGatherer()

#### Available Data Sources

In [ ]:
dg.available_sources()

#### Fetching Data from EODHD

In [ ]:
eodhd_data = dg.get_sync('AAPL', source='eodhd')
eodhd_data.head()

#### Fetching Data from AlphaVantage

In [ ]:
alphavantage_data = dg.get_sync('GOOGL', source='alphavantage')
alphavantage_data.head()

#### Fetching Data from Yahoo Finance

In [ ]:
yahoo_data = dg.get_sync('TSLA', source='yahoo', period='1y')
yahoo_data.head()

#### Using a Custom Data Source

In [ ]:
custom_data = dg.get_sync('MSFT', source='custom')
custom_data.head()

#### Generating Synthetic Data

In [ ]:
# Generate data with normally distributed returns
normal_data = dg.generate_synthetic_sync(
    distribution_type='normal',
    periods=100,
    return_mean=0.001,
    return_std=0.02
)
normal_data.head()

In [ ]:
normal_data[["Volume"]].to_pandas().plot(kind="bar")

In [ ]:
# Generate data using a GARCH model
garch_data = dg.generate_synthetic_sync(
    distribution_type='garch',
    periods=100
)
garch_data.head()

In [ ]:
garch_data[["Volume"]].to_pandas().plot(kind="bar")